In [22]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime
import pickle
import matplotlib as plt

In [13]:

# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
training_set = train_datagen.flow_from_directory(
        'train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

Found 1314 images belonging to 2 classes.


In [18]:
test_set = test_datagen.flow_from_directory(
        'test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

Found 194 images belonging to 2 classes.


In [19]:
model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

D:\anaconda\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
83/83 [==============================] - 25s 296ms/step - loss: 0.5944 - accuracy: 0.6408 - val_loss: 0.2258 - val_accuracy: 0.9175
Epoch 2/10
83/83 [==============================] - 25s 295ms/step - loss: 0.1970 - accuracy: 0.9300 - val_loss: 0.1115 - val_accuracy: 0.9588
Epoch 3/10
83/83 [==============================] - 27s 324ms/step - loss: 0.1326 - accuracy: 0.9528 - val_loss: 0.0885 - val_accuracy: 0.9536
Epoch 4/10
83/83 [==============================] - 25s 294ms/step - loss: 0.1151 - accuracy: 0.9513 - val_loss: 0.0656 - val_accuracy: 0.9845
Epoch 5/10
83/83 [==============================] - 22s 265ms/step - loss: 0.0752 - accuracy: 0.9718 - val_loss: 0.0864 - val_accuracy: 0.9742
Epoch 6/10
83/83 [==============================] - 22s 265ms/step - loss: 0.0677 - accuracy: 0.9749 - val_loss: 0.0825 - val_accuracy: 0.9691
Epoch 7/10
83/83 [==============================] - 22s 260ms/step - loss: 0.0653 - accuracy: 0.9787 - val_loss: 0.0909 - val_accuracy: 0.9691

In [20]:
model.save('mymodelb.h5',model_saved)
model.save_weights('model_b_weights')

In [21]:
np.save('modelb.npy',model_saved.history)

In [47]:
filename = 'finalized_model.h5'
pickle.dump(model, open(filename, 'wb'))

TypeError: cannot pickle 'weakref' object

In [8]:
model_saved2 = load_model('mymodelnew.h5')

In [9]:
model_saved2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

In [14]:
import matplotlib.pyplot as plt
plt.figure(0)
plt.plot(model_saved2.history["accuracy"],label="training accuracy")
plt.plot(model_saved2.history["val_accuracy"],label="training val_accuracy")
plt.xlabel("epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("ACCURACY USING AIM")

TypeError: 'NoneType' object is not subscriptable

<Figure size 432x288 with 0 Axes>

In [16]:
plt.figure(0)
plt.plot(model_saved2.history["loss"],label="training loss")
plt.plot(model_saved2.history["val_accuracy"],label="training val_loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.title("loss USING AIM")

TypeError: 'NoneType' object is not subscriptable

<Figure size 432x288 with 0 Axes>

In [10]:

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [11]:
while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=model_saved2.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()